In [14]:
print("Discriminator Output Size:", real_outputs.size())
print("Real Images Size:", real_images.size())

Discriminator Output Size: torch.Size([2, 1, 5, 5])
Real Images Size: torch.Size([2, 1, 64, 64])


The architecture of the GAN implemented in the provided code is a DCGAN (Deep Convolutional Generative Adversarial Network). DCGAN is a popular and effective variant of GAN that uses deep convolutional neural networks for both the generator and discriminator.

DCGAN introduces several key architectural changes compared to the original GAN:

1. Convolutional Layers: The generator and discriminator use convolutional layers instead of fully connected layers, making them better suited for processing images.

2. Transposed Convolution (ConvTranspose2d): The generator uses transposed convolutional layers (also known as deconvolution) to upscale the random noise (latent vector) into a generated image.

3. Batch Normalization: Batch normalization is applied after each convolutional layer in both the generator and discriminator. This helps stabilize and accelerate training.

4. Leaky ReLU Activation: Leaky ReLU activation function is used in the discriminator, allowing gradients to flow through when the output is negative.

5. Sigmoid Activation: The generator's output is passed through a sigmoid activation function to ensure the generated image pixels are in the range [0, 1].

Overall, DCGANs have been found to be very effective for generating high-quality images and are widely used for various image synthesis tasks. The architecture is flexible and can be adapted to different datasets and image sizes by adjusting the network sizes, layers, and hyperparameters.

The error message you encountered indicates that there is not enough memory available to allocate the required tensors. The model architecture you are using, particularly the transposed convolutional layers, requires a significant amount of memory, especially for larger image sizes like 512x512.

To address this issue, you can try the following options:

1. Reduce the model's capacity: You can decrease the `hidden_size` in the `Generator` class or use fewer layers in the generator and discriminator to reduce the memory footprint.

2. Decrease the batch size: Using a smaller batch size in the training loop will require less memory. Try reducing the `batch_size` in the `DataLoader`.

3. Use GPU with larger memory: If you have access to a GPU with more memory, you can try running the code on that GPU.

4. Resize the input images: If reducing the batch size or model capacity is not feasible, you can resize the input images to a smaller size (e.g., 128x128) during preprocessing. However, this may affect the quality of the generated images.

5. Use mixed precision training: If you have a GPU with Tensor Cores (e.g., NVIDIA's Volta and later GPUs), you can use mixed precision training to reduce memory consumption.

Remember that working with large images in GANs can be computationally expensive and memory-intensive. If you're working with limited hardware resources, consider starting with smaller image sizes to test your implementation before scaling up to 512x512 images.

In [13]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
import torch
import torch.nn as nn
import torch.nn.functional as F

# Generator class
class Generator(nn.Module):
    def __init__(self, z_size, channels, hidden_size=64):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # Input: z_size x 1 x 1
            nn.ConvTranspose2d(z_size, hidden_size * 4, 4, 1, 0, bias=False),
            nn.BatchNorm2d(hidden_size * 4),
            nn.ReLU(True),
            # Output: (hidden_size*4) x 4 x 4

            nn.ConvTranspose2d(hidden_size * 4, hidden_size * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(hidden_size * 2),
            nn.ReLU(True),
            # Output: (hidden_size*2) x 8 x 8

            nn.ConvTranspose2d(hidden_size * 2, hidden_size, 4, 2, 1, bias=False),
            nn.BatchNorm2d(hidden_size),
            nn.ReLU(True),
            # Output: hidden_size x 16 x 16

            nn.ConvTranspose2d(hidden_size, channels, 4, 2, 1, bias=False),
            nn.Tanh()
            # Output: channels x 32 x 32 (final image size)
        )

    def forward(self, x):
        return self.main(x)


# Discriminator class
class Discriminator(nn.Module):
    def __init__(self, input_size, hidden_dim, output_size):
        super(Discriminator, self).__init__()
 
        self.fc1 = nn.Linear(input_size, hidden_dim*4)
        self.fc2 = nn.Linear(hidden_dim*4, hidden_dim*2)
        self.fc3 = nn.Linear(hidden_dim*2, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, output_size)
        self.dropout = nn.Dropout(0.3)
 
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = self.dropout(x)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = self.dropout(x)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = self.dropout(x)
        out = torch.sigmoid(self.fc4(x))  # Use sigmoid activation to ensure output in the range [0, 1]
        return out.view(-1, 1, 1, 1)  # Reshape the output to (batch_size, 1, 1, 1)



# Custom Dataset and transformation
class BrainTumorDataset(Dataset):
    def __init__(self, data_folder, transform=None):
        self.data_folder = data_folder
        self.image_list = os.listdir(data_folder)
        self.transform = transform

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        image_path = os.path.join(self.data_folder, self.image_list[idx])
        image = Image.open(image_path).convert("L")  # Convert to grayscale image
        if self.transform:
            image = self.transform(image)
        return image


data_folder = r'C:\Users\ADMIN\project soft computing\IPDataForGlioma'
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize to 64x64 for faster training
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),  # Normalize to range [-1, 1]
])
brain_tumor_dataset = BrainTumorDataset(data_folder, transform=transform)

# DataLoader
batch_size = 64  # Adjust batch size according to your preference and resources
train_loader = DataLoader(brain_tumor_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

# Assuming you are generating 512x512 RGB images
channels = 3
image_size = 512
input_size = channels * image_size * image_size

# Discriminator hyperparameters
d_hidden_size = 32
d_output_size = 1

# Generator hyperparameters
z_size = 100
g_hidden_size = 32
g_output_size = channels * image_size * image_size

# Instantiate the generator
G = Generator(z_size, g_hidden_size, g_output_size)

# Instantiate the discriminator
D = Discriminator(input_size, d_hidden_size, d_output_size)

# Optimizers
lr = 0.0002  # Reduce the learning rate for stability
g_optimizer = optim.Adam(G.parameters(), lr=lr, betas=(0.5, 0.999))
d_optimizer = optim.Adam(D.parameters(), lr=lr, betas=(0.5, 0.999))

# Training hyperparameters
num_epochs = 100  # Increase the number of epochs for better convergence
sample_size = 16
fixed_z = torch.randn(sample_size, z_size, 1, 1)

# Training loop
G.train()
D.train()
losses_g = []
losses_d = []
criterion = nn.BCELoss()

output_dir = r'C:\Users\ADMIN\project soft computing\new_generated_images_glioma'


# Determine the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the selected device
G.to(device)
D.to(device)

# Training loop
for epoch in range(num_epochs):
    for batch_i, real_images in enumerate(train_loader):
        batch_size = real_images.size(0)

        # Move the real images to the selected device
        real_images = real_images.to(device)

        # Train the discriminator with real images
        d_optimizer.zero_grad()
        real_labels = torch.ones((batch_size, 1, 1, 1), device=device)  # Labels for real images (1)
        real_outputs = D(real_images)
        d_loss_real = criterion(real_outputs, real_labels)
        d_loss_real.backward(retain_graph=True)

        # Train the discriminator with fake images generated by the generator
        z = torch.randn(batch_size, z_size, 1, 1, device=device)
        fake_images = G(z)
        fake_labels = torch.zeros((batch_size, 1, 1, 1), device=device)  # Labels for fake images (0)
        fake_outputs = D(fake_images.detach())  # Detach fake_images from the generator to prevent gradients from propagating to G
        d_loss_fake = criterion(fake_outputs, fake_labels)
        d_loss_fake.backward()


        # Calculate gradient penalty
        gradient_penalty = calculate_gradient_penalty(D, real_images, fake_images, device)
        gradient_penalty.backward()

        # Combine the losses and update discriminator's parameters
        d_loss = d_loss_real + d_loss_fake + gradient_penalty
        d_optimizer.step()

        # Train the generator
        g_optimizer.zero_grad()
        real_labels = torch.ones(batch_size, 1)  # Labels for fake images (1) since we want to fool the discriminator
        fake_outputs = D(fake_images)
        g_loss = criterion(fake_outputs, real_labels)
        g_loss.backward()
        g_optimizer.step()

        # Print the losses
        if batch_i % 100 == 0:
            print(f"Epoch [{epoch + 1}/{num_epochs}] | Batch [{batch_i + 1}/{len(train_loader)}] | Gen Loss: {g_loss.item():.4f} | Disc Loss: {d_loss.item():.4f}")

    # Save generator and discriminator losses for plotting
    losses_g.append(g_loss.item())
    losses_d.append(d_loss.item())

    # Generate and save sample images every few epochs
    if (epoch + 1) % 10 == 0:
        G.eval()
        with torch.no_grad():
            samples_z = G(fixed_z)
        # Rescale generated images back to the range [0, 1]
        samples_z = (samples_z + 1) / 2

        # Save the generated images
        for i, sample in enumerate(samples_z):
            image_path = os.path.join(output_dir, f"generated_image_epoch{epoch + 1}_sample{i + 1}.png")
            # Reshape the tensor to 2D (64x64) and convert to grayscale image
            sample_image = transforms.ToPILImage()(sample.view(64, 64).cpu())
            sample_image.save(image_path)

        G.train()       
        
# Plot the generator and discriminator losses
plt.plot(losses_g, label="Generator Loss")
plt.plot(losses_d, label="Discriminator Loss")
plt.title("Generator and Discriminator Losses")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()


RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 20132659200 bytes.

In [21]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
import torch
import torch.nn as nn
import torch.nn.functional as F

# Generator class
class Generator(nn.Module):
    def __init__(self, z_size, channels, hidden_size=16):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # Input: z_size x 1 x 1
            nn.ConvTranspose2d(z_size, hidden_size * 4, 4, 1, 0, bias=False),
            nn.BatchNorm2d(hidden_size * 4),
            nn.ReLU(True),
            # Output: (hidden_size*4) x 4 x 4

            nn.ConvTranspose2d(hidden_size * 4, hidden_size * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(hidden_size * 2),
            nn.ReLU(True),
            # Output: (hidden_size*2) x 8 x 8

            nn.ConvTranspose2d(hidden_size * 2, hidden_size, 4, 2, 1, bias=False),
            nn.BatchNorm2d(hidden_size),
            nn.ReLU(True),
            # Output: hidden_size x 16 x 16

            nn.ConvTranspose2d(hidden_size, channels, 4, 2, 1, bias=False),
            nn.Tanh()
            # Output: channels x 32 x 32 (final image size)
        )

    def forward(self, x):
        return self.main(x)


# Discriminator class
class Discriminator(nn.Module):
    def __init__(self, input_size, hidden_dim, output_size):
        super(Discriminator, self).__init__()
 
        self.fc1 = nn.Linear(input_size, hidden_dim*4)
        self.fc2 = nn.Linear(hidden_dim*4, hidden_dim*2)
        self.fc3 = nn.Linear(hidden_dim*2, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, output_size)
        self.dropout = nn.Dropout(0.3)
 
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = self.dropout(x)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = self.dropout(x)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = self.dropout(x)
        out = torch.sigmoid(self.fc4(x))  # Use sigmoid activation to ensure output in the range [0, 1]
        return out.view(-1, 1, 1, 1)  # Reshape the output to (batch_size, 1, 1, 1)



# Custom Dataset and transformation
class BrainTumorDataset(Dataset):
    def __init__(self, data_folder, transform=None):
        self.data_folder = data_folder
        self.image_list = os.listdir(data_folder)
        self.transform = transform

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        image_path = os.path.join(self.data_folder, self.image_list[idx])
        image = Image.open(image_path).convert("L")  # Convert to grayscale image
        if self.transform:
            image = self.transform(image)
        return image


data_folder = r'C:\Users\ADMIN\project soft computing\IPDataForGlioma'
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize to 64x64 for faster training
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),  # Normalize to range [-1, 1]
])
brain_tumor_dataset = BrainTumorDataset(data_folder, transform=transform)

# DataLoader
batch_size = 32  # Adjust batch size according to your preference and resources
train_loader = DataLoader(brain_tumor_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

# Assuming you are generating 512x512 RGB images
channels = 3
image_size = 512
input_size = channels * image_size * image_size

# Discriminator hyperparameters
hidden_dim = 16
d_output_size = 1

# Generator hyperparameters
z_size = 100
g_hidden_size = 16
g_output_size = channels * image_size * image_size

# Instantiate the generator
G = Generator(z_size, g_hidden_size, g_output_size)

# Instantiate the discriminator
D = Discriminator(input_size, hidden_dim, d_output_size)

# Optimizers
lr = 0.0002  # Reduce the learning rate for stability
g_optimizer = optim.Adam(G.parameters(), lr=lr, betas=(0.5, 0.999))
d_optimizer = optim.Adam(D.parameters(), lr=lr, betas=(0.5, 0.999))

# Training hyperparameters
num_epochs = 100  # Increase the number of epochs for better convergence
sample_size = 16
fixed_z = torch.randn(sample_size, z_size, 1, 1)

# Training loop
G.train()
D.train()
losses_g = []
losses_d = []
criterion = nn.BCELoss()

output_dir = r'C:\Users\ADMIN\project soft computing\new_generated_images_glioma'


# Determine the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the selected device
G.to(device)
D.to(device)

# Training loop
for epoch in range(num_epochs):
    for batch_i, real_images in enumerate(train_loader):
        batch_size = real_images.size(0)

        # Move the real images to the selected device
        real_images = real_images.to(device)

        # Train the discriminator with real images
        d_optimizer.zero_grad()
        real_labels = torch.ones((batch_size, 1, 1, 1), device=device)  # Labels for real images (1)
        real_outputs = D(real_images)
        d_loss_real = criterion(real_outputs, real_labels)
        d_loss_real.backward(retain_graph=True)

        # Train the discriminator with fake images generated by the generator
        z = torch.randn(batch_size, z_size, 1, 1, device=device)
        fake_images = G(z)
        fake_labels = torch.zeros((batch_size, 1, 1, 1), device=device)  # Labels for fake images (0)
        fake_outputs = D(fake_images.detach())  # Detach fake_images from the generator to prevent gradients from propagating to G
        d_loss_fake = criterion(fake_outputs, fake_labels)
        d_loss_fake.backward()


        # Calculate gradient penalty
        gradient_penalty = calculate_gradient_penalty(D, real_images, fake_images, device)
        gradient_penalty.backward()

        # Combine the losses and update discriminator's parameters
        d_loss = d_loss_real + d_loss_fake + gradient_penalty
        d_optimizer.step()

        # Train the generator
        g_optimizer.zero_grad()
        real_labels = torch.ones(batch_size, 1)  # Labels for fake images (1) since we want to fool the discriminator
        fake_outputs = D(fake_images)
        g_loss = criterion(fake_outputs, real_labels)
        g_loss.backward()
        g_optimizer.step()

        # Print the losses
        if batch_i % 100 == 0:
            print(f"Epoch [{epoch + 1}/{num_epochs}] | Batch [{batch_i + 1}/{len(train_loader)}] | Gen Loss: {g_loss.item():.4f} | Disc Loss: {d_loss.item():.4f}")

    # Save generator and discriminator losses for plotting
    losses_g.append(g_loss.item())
    losses_d.append(d_loss.item())

    # Generate and save sample images every few epochs
    if (epoch + 1) % 10 == 0:
        G.eval()
        with torch.no_grad():
            samples_z = G(fixed_z)
        # Rescale generated images back to the range [0, 1]
        samples_z = (samples_z + 1) / 2

        # Save the generated images
        for i, sample in enumerate(samples_z):
            image_path = os.path.join(output_dir, f"generated_image_epoch{epoch + 1}_sample{i + 1}.png")
            # Reshape the tensor to 2D (64x64) and convert to grayscale image
            sample_image = transforms.ToPILImage()(sample.view(64, 64).cpu())
            sample_image.save(image_path)

        G.train()       
        
# Plot the generator and discriminator losses
plt.plot(losses_g, label="Generator Loss")
plt.plot(losses_d, label="Discriminator Loss")
plt.title("Generator and Discriminator Losses")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()


RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 20132659200 bytes.

In [1]:
!pip install swat

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 54.1/54.1 MB 925.3 kB/s eta 0:00:00


In [3]:
import swat

In [8]:
conn = swat.CAS(hostname="https://www.sas.com/en_in/trials/software/viya/viya-trial-form.html",
              port=5570, username="Pratik bharadwaj", password="Singh@9939")

SWATError: Could not find a suitable TLS CA certificate bundle, invalid path: C:\Program Files\Common Files\ssl/cert.pem

In [11]:
conn = swat.CAS(hostname="azureuse011696.my-trials.sas.com",  # Replace with your SAS Viya URL
                port=11696,  # Default CAS port for SAS Viya
                username="Pratik bharadwaj",
                password="Singh@9939")


ERROR: Failed to connect to host 'azureuse011696.my-trials.sas.com', port 11696.


SWATError: Could not connect to 'azureuse011696.my-trials.sas.com' on port 11696.